In [19]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bannu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bannu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
class AVML:
    import pandas as pd 
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import nltk
    import string
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    
    nltk.download('stopwords')
    nltk.download('punkt')
    
    def __init__(self):
        self.cv = CountVectorizer( stop_words='english' )
        self.vectors = None
        self.df = None
        self.ps = PorterStemmer() 

    def train(self,df):
        df['words'] = df[['Herb' , 'Benefits' ,'Medicinal_Uses']].apply(lambda row :', '.join(row.values.astype(str)),axis=1)
        df['words'] = df['words'].apply(self.transform_text)
    
        self.df =df
        self.vectors = self.cv.fit_transform(self.df['words']).toarray()   

    def recommend(self,input ,top_n=3):
        trans_input = self.transform_text(input)
        vector_input = self.cv.transform([trans_input])

        similarity_scores = cosine_similarity(vector_input ,self.vectors).flatten()
        top_indx = similarity_scores.argsort()[-top_n:][::-1]
        
        recomend = self.df.iloc[top_indx].reset_index()
        similarity = similarity_scores[top_indx]
        return recomend , similarity
        

    def transform_text(self,text):
        text = text.lower()
        text = text.replace(r',', '')
        text = nltk.word_tokenize(text)
        
        y = []
    
        for i in text:
            if i not in stopwords.words('english') and i not in string.punctuation:
                y.append(i)
             
        text = y[:]
        y.clear()
        for i in text:
            y.append(self.ps.stem(i))
        
        return ' '.join(y)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bannu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bannu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
df = pd.read_csv('data.csv')

model  = AVML()
model.train(df)

In [25]:
pred , similarity = model.recommend('i have constipation ', top_n=2)
print("        RECOMMENDATION              \n")
for i in range(pred.shape[0]):
    print("*"*30)
    print()
    print(f"Herb {i+1} : {pred['Herb'][i]} ")
    print(f'S_Score : {similarity[i].round(2)}')
    print()
print("*"*30)

        RECOMMENDATION              

******************************

Herb 1 : Haritaki 
S_Score : 0.33

******************************

Herb 2 : Bael 
S_Score : 0.28

******************************


In [23]:
import pickle

# Train your model first
df = pd.read_csv('data.csv')
model = AVML()
model.train(df)

# Save the model to a file using pickle
with open('avml_model.pkl', 'wb') as file:
    pickle.dump(model, file)
